We will be researching the Cart-pole problem with a policy gradient agorithim

We Must first build our enviorment that are reinforcement learning model will learn in

In [1]:
import mujoco
import mujoco_viewer

Necessary Constants for how we want the learning process to undergo

In [2]:
RENDER_TRAINING = True

In [2]:
def create_env(model_path="cartpole.xml"):
    model = mujoco.MjModel.from_xml_path(model_path)
    data = mujoco.MjData(model)
    if RENDER_TRAINING:
        viewer = mujoco_viewer.MujocoViewer(model, data)
    else:
        viewer = None
    return model, data, viewer

Now let's create the neccesary functions for our enviorment

Our model will take in 4 values: X value, X velocity, Theta value, Theta velocity.

We will need the enviorment randomized every new episode played by the model

In [3]:
import random as rand
import math
def Intial_state():
    theta = rand.uniform(-math.radians(6), math.radians(6))
    xpos = rand.uniform(-0.05, 0.05)
    xdot = rand.uniform(-0.5, 0.5)
    theta_dot = 0
    return xpos, xdot, theta, theta_dot

Now lets make the neccessary functions for our model. It needs to pushed to the left and to the right. We will use the data.crtl[0] to use the simulated acctuators that are in our model. The data element is all of the data of the cartpole at any moment this includes X Velo, theta Velo, X Pos, Theta Pos

In [4]:
import numpy as np
FORCE_MAG = 1.0

def right(data, force=FORCE_MAG):
    data.ctrl[0] = np.clip(force, -1.0, 1.0)


def left(data, force=FORCE_MAG):
    data.ctrl[0] = np.clip(-force, -1.0, 1.0)

Our model will need to know the state it is in, so we will create a function to get the raw values of the state for the model

In [5]:
def gather_raw_state(data):
    xpos = data.qpos[0]
    xdot = data.qvel[0]
    theta = data.qpos[1]
    theta_dot = data.qvel[1]
    return np.array([xpos, xdot, theta, theta_dot], dtype=np.float32)

Now we must create our Nueral Network for the cartpole problem
We will be creating a nueral netowrk through pytorch. It will take in 4 dimensions as an input for the network: X Value, X Velocity, Theta value, Theta Velocity

We will have 3 layers in the netowrk, teh first will take in the state layer, second being an input from the frist, and the thrid being an input from the second and outputting an action from our action set. Our action set is either sliding to the left or sliding to the right.

In [ ]:
import torch
import torch.nn as nn

class PolicyNetwork(nn.Module):
    def __init__(self,state_dim = 4, action_dim = 2, hidden_dim = 128):
        super().__init__()
        self.layer1 = nn.Linear(state_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.softmax(self.layer2(x), dim=-1)
        return self.output_layer(x)